In [3]:
pip install langchain-groq


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama-3.3-70b-versatile"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, as part of the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [9]:
%pip install -qU langchain_community beautifulsoup4


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [10]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.atlassian.com/company/careers/details/17193")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.



         Job Details | Atlassian                                  Close   View this page in your language?   All languages   Choose your language    中文   Deutsch   English   Español   Français   Italiano   한국어   Nederlands   日本語   Português   Pусский   Polski                            Get it free     Products    Featured    Developers    Product Managers    IT professionals    Business Teams    Leadership Teams       Featured   Developers   Product Managers   IT professionals   Business Teams   Leadership Teams    See all products       Featured            Jira  Flexible project management             Confluence  Knowledge, all in one place             Jira Service Management  High-velocity service delivery             Trello  Organized & visualized work             Rovo NEW  Unlock enterprise knowledge             Jira Product Discovery NEW  Capture & prioritize ideas             Compass NEW  Optimize software health             Guard NEW  Enhanced cloud security             Loom NE

In [21]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [24]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Senior Data Engineer',
 'experience': 'At least 5+ years professional experience as a Sr. Software Engineer or Sr. Data Engineer',
 'skills': ['Python',
  'Java',
  'Scala',
  'SQL',
  'Data modeling',
  'Data warehousing',
  'Spark',
  'Hive',
  'Airflow',
  'Amazon Web Services',
  'EMR',
  'Kinesis',
  'RDS',
  'S3',
  'SQS'],
 'description': 'Atlassian is looking for a Senior Data Engineer to join our Go-To Market Data Engineering (GTM-DE) team which is responsible for building our data lake, maintaining our big data pipelines / services and facilitating the movement of billions of messages each day.'}

In [25]:
type(json_res)

dict

In [27]:
pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 55.7 MB/s eta 0:00:0000:01

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [28]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [29]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [30]:
job = json_res
job['skills']

['Python',
 'Java',
 'Scala',
 'SQL',
 'Data modeling',
 'Data warehousing',
 'Spark',
 'Hive',
 'Airflow',
 'Amazon Web Services',
 'EMR',
 'Kinesis',
 'RDS',
 'S3',
 'SQS']

In [31]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/kotlin-android-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/kotlin-android-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/flutter-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/ios-ar-portfoli

In [32]:
job

{'role': 'Senior Data Engineer',
 'experience': 'At least 5+ years professional experience as a Sr. Software Engineer or Sr. Data Engineer',
 'skills': ['Python',
  'Java',
  'Scala',
  'SQL',
  'Data modeling',
  'Data warehousing',
  'Spark',
  'Hive',
  'Airflow',
  'Amazon Web Services',
  'EMR',
  'Kinesis',
  'RDS',
  'S3',
  'SQS'],
 'description': 'Atlassian is looking for a Senior Data Engineer to join our Go-To Market Data Engineering (GTM-DE) team which is responsible for building our data lake, maintaining our big data pipelines / services and facilitating the movement of billions of messages each day.'}

In [33]:
job = json_res
job['skills']

['Python',
 'Java',
 'Scala',
 'SQL',
 'Data modeling',
 'Data warehousing',
 'Spark',
 'Hive',
 'Airflow',
 'Amazon Web Services',
 'EMR',
 'Kinesis',
 'RDS',
 'S3',
 'SQS']

In [35]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Michal, a business development executive at Startup X. X is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of X
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase X portfolio: {link_list}
        Remember you are Michal, BDE at X. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Data Engineering Solutions for Atlassian's Go-To Market Team

Dear Hiring Manager,

I came across the Senior Data Engineer role at Atlassian and was impressed by the company's commitment to building a robust data lake and maintaining big data pipelines. As a Business Development Executive at Startup X, I'd like to introduce our company as a potential partner in fulfilling your data engineering needs.

With over 5+ years of experience in delivering tailored solutions, our team at Startup X has empowered numerous enterprises with scalable, optimized, and efficient data engineering services. Our expertise in Python, Java, Scala, SQL, data modeling, data warehousing, Spark, Hive, Airflow, and Amazon Web Services (including EMR, Kinesis, RDS, S3, and SQS) aligns perfectly with your requirements.

Our portfolio showcases our capabilities in:

* Machine learning and Python development: https://example.com/ml-python-portfolio
* Java-based solutions: https://example.com/java-por